# Freed RG Lab — receipts & plots

This notebook pulls receipts from `analysis/freed/` and `.tau_ledger/freed/`,
draws a couple of simple plots (if `matplotlib` is available), and prints a compact summary.


In [ ]:
import sys, os, glob, json, math
# Windows asyncio policy to placate nbconvert on Win10/11
if sys.platform.startswith('win'):
    import asyncio
    try:
        asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
    except Exception as e:
        print('[info] cannot set WindowsSelectorEventLoopPolicy:', e)
try:
    import matplotlib
    matplotlib.use('Agg')  # headless-safe
    import matplotlib.pyplot as plt
except Exception as e:
    plt = None; print('[info] matplotlib not available:', e)

def latest(pattern):
    files = sorted(glob.glob(pattern))
    return files[-1] if files else None

def load_json(path):
    try:
        with open(path, 'r', encoding='utf-8') as f:
            return json.load(f)
    except Exception as e:
        print('[warn] cannot load', path, e); return None

os.makedirs('analysis/freed', exist_ok=True)


## Determinant identity (trace vs finite difference)


In [ ]:
p = latest('analysis/freed/*_determinant_identity.json')
if not p:
    print('[warn] no determinant_identity receipt found')
else:
    j = load_json(p)
    xs  = [row['ell'] for row in j['grid']]
    lhs = [row['lhs_tr_identity'] for row in j['grid']]
    rhs = [row['rhs_fd'] for row in j['grid']]
    print('[determinant] max_abs_err =', j.get('max_abs_err'))
    if plt:
        import matplotlib.pyplot as plt  # ensure alias
        plt.figure()
        plt.plot(xs, lhs, label='trace identity')
        plt.plot(xs, rhs, linestyle='dashed', label='finite diff')
        plt.title('trace vs d/dℓ log det Σ')
        plt.legend(); plt.tight_layout(); plt.show()


## TMF deltas (mock elliptic correction)


In [ ]:
p = latest('analysis/freed/*_tmf_deltas.json')
if not p:
    print('[warn] no tmf_deltas receipt found (mock may be off)')
else:
    j = load_json(p) or {}
    tmf = j.get('tmf', {})
    print('[tmf] levels=', tmf.get('levels'), 'eps=', tmf.get('eps'))
    for k in ('phi_off','phi_on'):
        sec = tmf.get(k, {})
        print(f"[tmf {k}] delta_whole=", sec.get('delta_whole'), ' delta_segments=', sec.get('delta_segments'))
    print('[tmf] checks:', j.get('checks'))


## Defects (W(B₅) walls)


In [ ]:
p = latest('.tau_ledger/freed/defect_*.manifest.json')
if not p:
    print('[warn] no defect manifest found')
else:
    j = load_json(p) or {}
    cert = j.get('certificates', {})
    print('[defect] sign_flip_phase=', cert.get('sign_flip_phase'),
          'perm_phase=', cert.get('perm_phase'))
    print('[defect] sign_flip_shape_err=', cert.get('sign_flip_shape_err'),
          'perm_shape_err=', cert.get('perm_shape_err'))


## Axioms summary


In [ ]:
from pprint import pprint
p = latest('analysis/freed/axioms_*.json')
if not p:
    print('[warn] no axioms json found — run scripts/freed/axioms_table.py')
else:
    pprint(load_json(p))
